Project Edits based on meeting with Professor Mustafaraj

1. Make sure my second round of text classification is only on the confirmed people articles

In [1]:
import pandas as pd

In [21]:
df = pd.read_json("entity_results2.jsonl", lines=True)

In [22]:
df.head()

,QID,status,label,description,attributes,error_message
0,Q5296,success,Wikimedia main page,main page of a Wikimedia project,"{'instance of': 'Wikimedia internal item', 'su...",NaN
1,Q178995,success,HTTP cookie,small piece of data sent from a website and st...,"{'named after': 'cookie', 'Commons category': ...",NaN
2,Q23685,success,Jimmy Carter,president of the United States from 1977 to 19...,"{'Perlentaucher ID': 'jimmy-carter', 'given na...",NaN
3,Q866,success,YouTube,American video-sharing platform owned by Alpha...,"{'instance of': 'video streaming service', 'Co...",NaN
4,Q42253,success,URL,web address to a particular file or page,"{'subclass of': 'Uniform Resource Identifier',...",NaN


In [23]:
def combine_text(row):
    # Make sure attributes is a dictionary
    attributes = row["attributes"]
    if not isinstance(attributes, dict):
        attributes = {}
    
    # Get occupation safely
    occupation = attributes.get("occupation", "")
    
    # Combine text
    return f"{row['label']} {row['description']} {occupation}"


In [24]:
df["summary"] = df.apply(combine_text, axis=1)

In [25]:
df.head()

,QID,status,label,description,attributes,error_message,summary
0,Q5296,success,Wikimedia main page,main page of a Wikimedia project,"{'instance of': 'Wikimedia internal item', 'su...",NaN,Wikimedia main page main page of a Wikimedia p...
1,Q178995,success,HTTP cookie,small piece of data sent from a website and st...,"{'named after': 'cookie', 'Commons category': ...",NaN,HTTP cookie small piece of data sent from a we...
2,Q23685,success,Jimmy Carter,president of the United States from 1977 to 19...,"{'Perlentaucher ID': 'jimmy-carter', 'given na...",NaN,Jimmy Carter president of the United States fr...
3,Q866,success,YouTube,American video-sharing platform owned by Alpha...,"{'instance of': 'video streaming service', 'Co...",NaN,YouTube American video-sharing platform owned ...
4,Q42253,success,URL,web address to a particular file or page,"{'subclass of': 'Uniform Resource Identifier',...",NaN,URL web address to a particular file or page


In [27]:
df["instance_of"] = df["attributes"].apply(
    lambda x: x.get("instance of") if isinstance(x, dict) else None
)



In [28]:
df.head()

,QID,status,label,description,attributes,error_message,summary,instance_of
0,Q5296,success,Wikimedia main page,main page of a Wikimedia project,"{'instance of': 'Wikimedia internal item', 'su...",NaN,Wikimedia main page main page of a Wikimedia p...,Wikimedia internal item
1,Q178995,success,HTTP cookie,small piece of data sent from a website and st...,"{'named after': 'cookie', 'Commons category': ...",NaN,HTTP cookie small piece of data sent from a we...,None
2,Q23685,success,Jimmy Carter,president of the United States from 1977 to 19...,"{'Perlentaucher ID': 'jimmy-carter', 'given na...",NaN,Jimmy Carter president of the United States fr...,human
3,Q866,success,YouTube,American video-sharing platform owned by Alpha...,"{'instance of': 'video streaming service', 'Co...",NaN,YouTube American video-sharing platform owned ...,video streaming service
4,Q42253,success,URL,web address to a particular file or page,"{'subclass of': 'Uniform Resource Identifier',...",NaN,URL web address to a particular file or page,technical standard


In [29]:
humans_df = df[df["instance_of"] == "human"]


In [30]:
humans_df.head()

,QID,status,label,description,attributes,error_message,summary,instance_of
2,Q23685,success,Jimmy Carter,president of the United States from 1977 to 19...,"{'Perlentaucher ID': 'jimmy-carter', 'given na...",NaN,Jimmy Carter president of the United States fr...,human
7,Q322056,success,Lance Reddick,American actor (1962–2023),"{'Commons category': 'Lance Reddick', 'occupat...",NaN,Lance Reddick American actor (1962–2023) telev...,human
11,Q14752155,success,Pedro Pascal,Chilean-American actor (born 1975),"{'instance of': 'human', 'place of birth': 'Sa...",NaN,Pedro Pascal Chilean-American actor (born 1975...,human
13,Q193555,success,Brendan Fraser,Canadian-American actor (born 1968),"{'sex or gender': 'male', 'Commons category': ...",NaN,Brendan Fraser Canadian-American actor (born 1...,human
16,Q106997,success,Jamie Lee Curtis,American actress and author,"{'father': 'Tony Curtis', 'mother': 'Janet Lei...",NaN,Jamie Lee Curtis American actress and author a...,human


In [43]:
humans_df = humans_df.rename(columns={"QID": "qid"})

In [44]:
qidsdf = pd.read_csv("final-project-data2.csv")

In [45]:
qidsdf.head()

,date,country_code,article,qid,pageviews,lang_code
0,2023-03-15,US,Main_Page,Q5296,7132908,en
1,2023-03-16,US,Main_Page,Q5296,4532076,en
2,2023-03-01,US,Cookie_(informatique),Q178995,4251750,fr
3,2023-03-17,US,Main_Page,Q5296,4233371,en
4,2023-03-10,US,Cookie_(informatique),Q178995,4158637,fr


In [46]:
human_qids = set(humans_df["qid"])

In [47]:
qids_people_df = qidsdf[qidsdf["qid"].isin(human_qids)]

In [48]:
qids_people_df = qids_people_df.merge(
    humans_df[["qid", "instance_of", "summary"]],
    how="left"
)


In [60]:
qids_people_df.head()

,date,country_code,article,qid,pageviews,lang_code,instance_of,summary,category
0,2023-03-13,US,Jonathan_Ke_Quan,Q445017,805110,en,human,Ke Huy Quan American actor and stunt choreogra...,current
1,2023-03-13,US,Jamie_Lee_Curtis,Q106997,730692,en,human,Jamie Lee Curtis American actress and author a...,current
2,2023-03-17,US,Lance_Reddick,Q322056,690509,en,human,Lance Reddick American actor (1962–2023) telev...,current
3,2023-03-13,US,Brendan_Fraser,Q193555,671995,en,human,Brendan Fraser Canadian-American actor (born 1...,current
4,2023-03-18,US,Lance_Reddick,Q322056,630690,en,human,Lance Reddick American actor (1962–2023) telev...,current


In [50]:
import torch
import pandas as pd
from transformers import pipeline
from tqdm.notebook import tqdm

In [51]:
MODEL_NAME = "facebook/bart-large-mnli"
DEVICE = 0 if torch.cuda.is_available() else -1

print(f"Loading model: {MODEL_NAME} on device: {'GPU' if DEVICE == 0 else 'CPU'}")


Loading model: facebook/bart-large-mnli on device: CPU


In [52]:
classifier = pipeline(
    "zero-shot-classification",
    model=MODEL_NAME,
    device=DEVICE,
    batch_size=32
)

classifier

Device set to use cpu


In [53]:
labels = ["current", "historical"]

In [54]:
results = classifier(
        qids_people_df['summary'].tolist(),
        candidate_labels=labels,
        hypothesis_template= "This text is about {}.",
        multi_label=False
    )

print("Classification complete.")

Classification complete.


In [56]:
results

[{'sequence': 'Ke Huy Quan American actor and stunt choreographer actor',
  'labels': ['current', 'historical'],
  'scores': [0.6739802956581116, 0.32601967453956604]},
 {'sequence': 'Jamie Lee Curtis American actress and author actor',
  'labels': ['current', 'historical'],
  'scores': [0.7532625794410706, 0.24673743546009064]},
 {'sequence': 'Lance Reddick American actor (1962–2023) television actor',
  'labels': ['current', 'historical'],
  'scores': [0.5642034411430359, 0.43579649925231934]},
 {'sequence': 'Brendan Fraser Canadian-American actor (born 1968) film actor',
  'labels': ['current', 'historical'],
  'scores': [0.7197417616844177, 0.2802582383155823]},
 {'sequence': 'Lance Reddick American actor (1962–2023) television actor',
  'labels': ['current', 'historical'],
  'scores': [0.5642034411430359, 0.43579649925231934]},
 {'sequence': 'Michelle Yeoh Malaysian actress actor',
  'labels': ['current', 'historical'],
  'scores': [0.7944908142089844, 0.205509215593338]},
 {'sequ

In [57]:
qids_people_df["category"] = [r['labels'][0] for r in results]

In [61]:
qids_people_df.head()

,date,country_code,article,qid,pageviews,lang_code,instance_of,summary,category
0,2023-03-13,US,Jonathan_Ke_Quan,Q445017,805110,en,human,Ke Huy Quan American actor and stunt choreogra...,current
1,2023-03-13,US,Jamie_Lee_Curtis,Q106997,730692,en,human,Jamie Lee Curtis American actress and author a...,current
2,2023-03-17,US,Lance_Reddick,Q322056,690509,en,human,Lance Reddick American actor (1962–2023) telev...,current
3,2023-03-13,US,Brendan_Fraser,Q193555,671995,en,human,Brendan Fraser Canadian-American actor (born 1...,current
4,2023-03-18,US,Lance_Reddick,Q322056,630690,en,human,Lance Reddick American actor (1962–2023) telev...,current


In [58]:
topresults = qids_people_df.groupby(['article', "country_code", "category"])['pageviews'].sum().reset_index()

In [59]:
topresults.head()

,article,country_code,category,pageviews
0,6ix9ine,GB,current,13235
1,6ix9ine,US,current,156024
2,A$AP_Rocky,DE,current,14453
3,A._R._Rahman,IN,current,10769
4,ASAP_Rocky,US,current,25047


In [62]:
topUS = topresults[topresults['country_code'].str.contains("US", case=False, na=False)]
topJP = topresults[topresults['country_code'].str.contains("JP", case=False, na=False)]
topGB = topresults[topresults['country_code'].str.contains("GB", case=False, na=False)]
topIN = topresults[topresults['country_code'].str.contains("IN", case=False, na=False)]
topDE = topresults[topresults['country_code'].str.contains("DE", case=False, na=False)]

In [63]:
topUS.sort_values(by='pageviews', ascending=False, inplace=True)
topJP.sort_values(by='pageviews', ascending=False, inplace=True)
topGB.sort_values(by='pageviews', ascending=False, inplace=True)
topIN.sort_values(by='pageviews', ascending=False, inplace=True)
topDE.sort_values(by='pageviews', ascending=False, inplace=True)

/var/folders/5q/q02qx9t97hdgr20tw_hpw2zw0000gn/T/ipykernel_72171/155755231.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  topUS.sort_values(by='pageviews', ascending=False, inplace=True)
/var/folders/5q/q02qx9t97hdgr20tw_hpw2zw0000gn/T/ipykernel_72171/155755231.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  topJP.sort_values(by='pageviews', ascending=False, inplace=True)
/var/folders/5q/q02qx9t97hdgr20tw_hpw2zw0000gn/T/ipykernel_72171/155755231.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/panda

In [64]:
topUS = topUS.head(25)
topJP = topJP.head(25)
topGB = topGB.head(25)
topIN = topIN.head(25)
topDE = topDE.head(25)

In [65]:
tophumans = pd.concat([topUS, topJP, topGB, topIN, topDE], ignore_index=True)

In [66]:
tophumans.to_csv("top25articles_humans2.csv", index=False)

In [68]:
qids_people_df.to_csv("human_classification2.csv", index=False)

In [70]:
qids_people_df['date'] = pd.to_datetime(qids_people_df['date'])

In [71]:
# Group by date and predicted_category
results_pct = qids_people_df.groupby(['date', 'category', 'country_code'])['pageviews'].sum().reset_index()

results_pct['date'] = pd.to_datetime(results_pct['date'])


# Pivot so that each category is a column
merged_daily_category_pivot = results_pct.pivot(index=['date', 'country_code'], columns='category', values='pageviews').fillna(0)

# Calculate percentages
merged_daily_category_pct = merged_daily_category_pivot.div(merged_daily_category_pivot.sum(axis=1), axis=0)


In [72]:
merged_daily_category_pct.to_csv("human_classification_daily_category_pct2.csv")

Now I can work on creating a visualization about the langauges present in each wikipedia

In [73]:
qidsdf.head()

,date,country_code,article,qid,pageviews,lang_code
0,2023-03-15,US,Main_Page,Q5296,7132908,en
1,2023-03-16,US,Main_Page,Q5296,4532076,en
2,2023-03-01,US,Cookie_(informatique),Q178995,4251750,fr
3,2023-03-17,US,Main_Page,Q5296,4233371,en
4,2023-03-10,US,Cookie_(informatique),Q178995,4158637,fr


In [74]:
US_langs = qidsdf[qidsdf['country_code'].str.contains("US", case=False, na=False)]
JP_langs = qidsdf[qidsdf['country_code'].str.contains("JP", case=False, na=False)]
GB_langs = qidsdf[qidsdf['country_code'].str.contains("GB", case=False, na=False)]
IN_langs = qidsdf[qidsdf['country_code'].str.contains("IN", case=False, na=False)]
DE_langs = qidsdf[qidsdf['country_code'].str.contains("DE", case=False, na=False)]

In [77]:
US_lang_counts = US_langs['lang_code'].value_counts()

In [108]:
US_lang_counts

lang_code
en    2876
ja      33
fr      31
es      19
de      15
ru      10
it       6
zh       4
vi       2
uk       2
nl       1
th       1
Name: count, dtype: int64

In [109]:
US_lang_counts_df = US_lang_counts.reset_index()   
US_lang_counts_df.columns = ['Language', 'Count']

In [110]:
US_lang_counts_df

,Language,Count
0,en,2876
1,ja,33
2,fr,31
3,es,19
4,de,15
5,ru,10
6,it,6
7,zh,4
8,vi,2
9,uk,2


In [111]:
US_lang_counts_df_condensed = US_lang_counts_df.copy()

In [120]:
US_lang_counts_df_condensed['Language'] = US_lang_counts_df_condensed['Language'].apply(lambda x: x if x == 'en' else 'Other')


In [121]:
US_lang_counts_df_condensed

,Language,Count
0,Other,15
1,en,2876
2,Other,19
3,Other,31
4,Other,6
5,Other,33
6,Other,1
7,Other,10
8,Other,1
9,Other,2


In [122]:
US_lang_counts_df_condensed = US_lang_counts_df_condensed.groupby('Language', as_index=False)['Count'].sum()

In [123]:
US_lang_counts_df_condensed

,Language,Count
0,Other,124
1,en,2876


In [81]:
import plotly.express as px

In [124]:
fig = px.bar(US_lang_counts_df_condensed, x='Language', y='Count',
             title='Number of Articles per Language',
             labels={'Language': 'Language', 'Count': 'Number of Articles'},
             color='Language')

fig.show()

In [89]:
JP_lang_counts = JP_langs['lang_code'].value_counts()
GB_lang_counts = GB_langs['lang_code'].value_counts()
IN_lang_counts = IN_langs['lang_code'].value_counts()
DE_lang_counts = DE_langs['lang_code'].value_counts()

In [90]:
JP_lang_counts_df = JP_lang_counts.reset_index()   
JP_lang_counts_df.columns = ['Language', 'Count']

GB_lang_counts_df = GB_lang_counts.reset_index()   
GB_lang_counts_df.columns = ['Language', 'Count']

IN_lang_counts_df = IN_lang_counts.reset_index()   
IN_lang_counts_df.columns = ['Language', 'Count']

DE_lang_counts_df = DE_lang_counts.reset_index()   
DE_lang_counts_df.columns = ['Language', 'Count']

In [139]:
JP_lang_counts_df

,Language,Count
0,ja,2958
1,en,37
2,tr,2
3,fr,1
4,vi,1
5,zh,1


In [92]:
GB_lang_counts_df

,Language,Count
0,en,2884
1,commons,31
2,fr,29
3,ja,27
4,de,12
5,tr,8
6,it,6
7,nl,2
8,vi,1


In [93]:
IN_lang_counts_df

,Language,Count
0,en,2901
1,hi,56
2,mai,21
3,mr,8
4,tr,6
5,fr,2
6,bn,2
7,vi,2
8,te,1
9,id,1


In [94]:
DE_lang_counts_df

,Language,Count
0,de,2738
1,en,118
2,ru,42
3,commons,31
4,fr,25
5,tr,11
6,zh,11
7,ja,9
8,vi,4
9,bg,4


In [140]:
JP_lang_counts_df_condensed = JP_lang_counts_df.copy()

GB_lang_counts_df_condensed = GB_lang_counts_df.copy()

IN_lang_counts_df_condensed = IN_lang_counts_df.copy()

DE_lang_counts_df_condensed = DE_lang_counts_df.copy()

In [141]:
JP_lang_counts_df_condensed['Language'] = JP_lang_counts_df_condensed['Language'].apply(lambda x: x if x == 'ja' else 'Other')

GB_lang_counts_df_condensed['Language'] = GB_lang_counts_df_condensed['Language'].apply(lambda x: x if x == 'en' else 'Other')

IN_lang_counts_df_condensed['Language'] = IN_lang_counts_df_condensed['Language'].apply(lambda x: x if x == 'en' else 'Other')

DE_lang_counts_df_condensed['Language'] = DE_lang_counts_df_condensed['Language'].apply(lambda x: x if x == 'de' else 'Other')



In [142]:
JP_lang_counts_df_condensed = JP_lang_counts_df_condensed.groupby('Language', as_index=False)['Count'].sum()

GB_lang_counts_df_condensed = GB_lang_counts_df_condensed.groupby('Language', as_index=False)['Count'].sum()

IN_lang_counts_df_condensed = IN_lang_counts_df_condensed.groupby('Language', as_index=False)['Count'].sum()

DE_lang_counts_df_condensed = DE_lang_counts_df_condensed.groupby('Language', as_index=False)['Count'].sum()

In [143]:
JP_lang_counts_df_condensed

,Language,Count
0,Other,42
1,ja,2958


Now I can load everything to csv files. I know this is a lot of files, but I don't have enough time to streamline all of this, so I am just going to do it this way

In [ ]:
US_lang_counts_df.to_csv("US_lang_counts_df.csv", index=False)

JP_lang_counts_df.to_csv("JP_lang_counts_df.csv", index=False)

GB_lang_counts_df.to_csv("GB_lang_counts_df.csv", index=False)

IN_lang_counts_df.to_csv("IN_lang_counts_df.csv", index=False)

DE_lang_counts_df.to_csv("DE_lang_counts_df.csv", index=False)

In [145]:
US_lang_counts_df_condensed.to_csv("US_lang_counts_df_condensed.csv", index=False)

JP_lang_counts_df_condensed.to_csv("JP_lang_counts_df_condensed.csv", index=False)

GB_lang_counts_df_condensed.to_csv("GB_lang_counts_df_condensed.csv", index=False)

IN_lang_counts_df_condensed.to_csv("IN_lang_counts_df_condensed.csv", index=False)

DE_lang_counts_df_condensed.to_csv("DE_lang_counts_df_condensed.csv", index=False)

In [150]:
US_lang_counts_df_condensed['Country'] = 'US'
JP_lang_counts_df_condensed['Country'] = 'JP'
GB_lang_counts_df_condensed['Country'] = 'GB'
IN_lang_counts_df_condensed['Country'] = 'IN'
DE_lang_counts_df_condensed['Country'] = 'DE'

In [151]:
US_lang_counts_df_condensed

,Language,Count,Country
0,Other,124,US
1,en,2876,US


In [153]:
combined_df = pd.concat([
        US_lang_counts_df_condensed,
        JP_lang_counts_df_condensed,
        GB_lang_counts_df_condensed,
        IN_lang_counts_df_condensed,
        DE_lang_counts_df_condensed
    ], ignore_index=True)

In [154]:
combined_df.to_csv("combined_df_condensed.csv", index=False)

In [155]:
US_lang_counts_df_condensed

,Language,Count,Country
0,Other,124,US
1,en,2876,US
